In [76]:
import pandas as pd
import numpy as np
import time

from numpy import dot
from numpy.linalg import norm

# Utils

In [77]:
def get_ratings_by_item(
    df, users_df, items, rating_column='Rating', user_column='UserId', item_column='ItemId'):
    
    aux = df.loc[df[item_column].isin(items)]
    aux = aux.pivot_table(index=user_column, columns=item_column, values=rating_column)
    users_df = users_df.merge(aux, how='left', on=[user_column])
    users_df = users_df.fillna('0')

    return users_df

In [78]:
def get_items_by_users(
    df, users, user_column='UserId', item_column='ItemId'):
    """
    Get items id evaluates by user target.

    Args:
      df: ps.DataFrame
          predict rating value
      user: str
          user id
      user_column: str, default UserId
          name of column with users id
      item_column: str, default ItemId
      
    Returns: 
    
    list of strings with all items ids evaluated by user target
    """
    aux = df.copy()
    items = [list(aux.loc[aux[user_column] == user][item_column]) for user in users]
    items = sorted(set(sum(items, [])))
    
    return items

In [79]:
def get_common_users_items(
    df, item, user_column='UserId', item_column='ItemId'):
    """
    Get the ids of users that evaluate a item.

    Args:
      df: ps.DataFrame
          predict rating value
      item: str
          item id
      user_column: str, default UserId
          name of column with users id
      item_column: str, default ItemId
          name of column with items id
    Returns: 
    
    list of strings with all users ids that evaluate a target item
    """
    
    return list(df.loc[df[item_column] == item][user_column])

In [80]:
def cosine_similarity(x, y):
    """
    Calculates cosine similarity between two vectors.

    Args:
      x: np.array
         vector of values
      y: np.array
         vector of valuesitem id
         
    Returns: 
    
    cosine similarity between x and y
    """
    
    return np.dot(x, y)/(norm(x)*norm(y))

# Item-based collaborative filtering

Users who like an item tend to like similar items

In [81]:
def combinations(items):
    
    comb = []
    k = 0
    for i in items:
        for j in range(1 + k, len(items)):
            comb.append((i, items[j]))
        k+=1

    return comb

In [82]:
def get_item_vector(item, users_df, ratings_df):
    
    aux = ratings_df.loc[ratings_df['ItemId'] == item][['UserId', 'Rating']]
    df = users_df.merge(aux, how='left', on=['UserId']).fillna(0)
    
    return df

In [83]:
def pairwise_item_sim(comb, users_df, ratings_df):
    
    similarities = []
    
    for i in range(len(comb)):
    
        i1 = get_item_vector(comb[i][0], users_df, ratings_df)['Rating']
        i2 = get_item_vector(comb[i][1], users_df, ratings_df)['Rating']

        similarities.append(cosine_similarity(i1, i2))
        
    df = pd.DataFrame({"items": comb, "similarity": similarities})

    return df

In [84]:
def get_item_neighbors(item_sim, item_target, ratings_df):
    
    common_users = get_common_users_items(ratings_df, item=item_target)
    
    if common_users == []:
        return None
        
    candidate_items = get_items_by_users(ratings_df, users=common_users)
    
    if candidate_items == []:
        return None
    
    neighbors = item_sim[item_sim['items'].apply(lambda x: item_target in x)]
    neighbors = neighbors.loc[neighbors['items'].apply(lambda x: any(candidate in x for candidate in candidate_items))]
    neighbors = neighbors.reset_index(drop=True)
    
    return neighbors

In [85]:
def calc_rmse(y_pred, y_true):
    """
    Calculate root mean squared error.

    Args:
      y_pred: float
          predict rating value
      y_true: float
          true rating value
    Returns: 
    
    rmse: float
        root mean squared error between predict rating values and true ratings values
    """
    
    return np.sqrt(((y_pred - y_true) ** 2).mean())

In [86]:
def calc_mae(y_pred, y_true):
    """
    Calculate mean absolute error.

    Args:
      y_pred: float
          predict rating value
      y_true: float
          true rating value
    
    Returns: 
    
    mae: float
        mean absolute error between predict rating values and true ratings values
    """
    return np.absolute(y_pred - y_true).mean()

In [87]:
def train_test_split(X, test_split=0.2):
    """
    Split dataset in train and test.

    Args:
      X: np.array
          ratings 
      test_split: float
          percentage of data for the test dataset 
    
    Returns: 
    
    train: np.array
        train dataset
    test:
        test dataset
    """
    
    size = int(X.shape[0])
    all_indexes = list(range(size))
                       
    indexes_test = list(np.random.choice(np.arange(0,size), int(size*test_split), replace=False))
    indexes_train = set(all_indexes) - set(indexes_test)
    
    train = X.loc[indexes_train].reset_index(drop=True)
    test = X.loc[indexes_test].reset_index(drop=True)
    
    return train, test

In [88]:
def clip(pred, max_pred=5, min_pred=1):
    """
    Clipping predict values in range [min_pred, max_pred].

    Args:
      pred : float
          predict value
      max_pred : int
          max value of rating
      min_pred : int
          min value of rating

    Returns: 

    pred: int
        predict value cliped
    """
    
    pred = max_pred if pred > max_pred else pred
    pred = min_pred if pred < min_pred else pred
    
    return pred

In [89]:
def update_weights(
    pu, qi, user, item, eui, num_factors, alpha=0.1, lamb=0.02):
    """
    Update user and item matrix weights.

    Args:
      pu: np.array
          user matrix
      qi: np.array
          item matrix
      user: str
          user code
      item: str
          item code
      eui: float
          predict error
      num_factor: int
          number of latent factors
      alpha: float
          learning rate

    Returns: 

    pu: np.array
        user matrix updated
    qi: np.array
          item matrix updated
    """
    
    for k in range(num_factors):
        
        puf = alpha * (eui * qi[k, item] - lamb * pu[user, k])
        qif = alpha * (eui * pu[user, k] - lamb * qi[k, item]) 
        
        pu[user, k] +=  puf
        qi[k, item] +=  qif       
            
    return pu, qi

In [90]:
def update_bias(
    bu, bi, user, item, eui, alpha, lamb):
    """"
    Update bias weights.

    Args:
      pu: np.array
          user matrix
      qi: np.array
          item matrix
      user: str
          user code
      item: str
          item code
      eui: float
          predict error
      num_factor: int
          number of latent factors
      alpha: float
          learning rate

    Returns: 

    pu: np.array
        user matrix updated
    qi: np.array
          item matrix updated
    """
    
    buf = alpha * (eui * bi[item] - lamb * bu[user])
    bif = alpha * (eui * bu[user] - lamb * bi[item])
    
    bu[user] += buf
    bi[item] += bif
    
    return bu, bi

In [91]:
def one_svd_predict(
    pu, qi, bu, bi, user, item, num_factors, mean):
    """
    Predict rating to a pair user item.

    Args:
      pu: np.array
          user matrix
      qi: np.array
          item matrix
      user: str
          user code
      item: str
          item code
      num_factor: int
          number of latent factors
      mean: float
          mean ratings of all users

    Returns: 

    float: predicted value      
    """
    
    return sum([pu[user, k] * qi[k, item] for k in range(num_factors)]) + mean + bu[user] + bi[item]

In [129]:
def svd_predict(
    X, pu, qi, bu, bi, num_factors, mean_ratings, max_pred=5, min_pred=1):
    """
    Predict rating for all pairs users items.

    Args:
      X: np.array
          rating matrix
      pu: np.array
          user matrix
      qi: np.array
          item matrix
      num_factors: int
          number of latent factors
      mean: float
          mean ratings of all users
      max_pred: int
          highest possible prediction
      min_pred: int
          lowest possible prediction

    Returns: 

    y_pred: np.array
        predict values
    y_true: np.array
        targets values
    """
    
    y_pred = []
    y_true = []
    
    for idx in range(X.shape[0]):
        
        user, item, rui = int(X[idx, 0]), int(X[idx, 1]), X[idx, 2]

        mean = mean_ratings.get(item)

        pred = one_svd_predict(pu, qi, bu, bi, user, item, num_factors, mean)
        
        y_pred.append(pred)
        y_true.append(rui)
    
    return np.asarray(y_pred), np.asarray(y_true)

In [147]:
def inicialization(n, m, num_factors):
    """
    Initializes weights.

    Args:
      n: int
          number of unique users in ratings data
      m: int
          number of unique itens in ratings data
      num_factors: int
          number of latent factors

    Returns: 

    pu: np.array
        user matrix random initialize
    qi: np.array
        item matrix random initialize
    bu: np.array
        bias vector for user weights initialize with zeros
    bi: np.array
        bias vector for item weights initialize with zeros
    """
    
    pu = np.random.normal(0, 0.2, (n, num_factors))
    qi = np.random.normal(0, 0.2, (m, num_factors))
    
    bu = np.zeros(n)
    bi = np.zeros(m)
    
    return pu, qi, bu, bi  

In [94]:
def scheduler(epoch, alpha):
    """
    Learning rating scheduler.
    
    Args:
      epoch: int
          actual epoch in training
      alpha: int
          learning rating

    Returns: 

    learning rate update 
    """
    
    if epoch < 2:
        return alpha
    else:
        return 0.05

In [128]:
def SGD(
    X, num_factors, n, m, mean_ratings, alpha=0.001, lamb=0.002):
    """
    Stochastic Gradiend Descent.

    Args:
      X: np.array
          ratings data 
      num_factor: int
          number of latent factors
      n: int
          number of unique users in ratings data
      m: int
          number of unique itens in ratings data
      alpha: float
          learning rate

    Returns: 

    pu: np.array
        user matrix factored
    qi: np.array
        item matrix factored
    bu: np.array
        bias vector for user weights
    bi: np.array
        bias vector for item weights
    """
    
    pu, qi, bu, bi = inicialization(n, m, num_factors)
    
    qi = qi.T
    
    for idx in range(X.shape[0]):
        
        user, item, rui = int(X[idx, 0]), int(X[idx, 1]), X[idx, 2]
        
        # get user mean ratings
        mean = mean_ratings.get(item)

        #predict rating
        pred = one_svd_predict(pu, qi, bu, bi, user, item, num_factors, mean)
        
        #calculate error
        eui = rui - pred
        
        #update bias
        bu, bi = update_bias(bu, bi, user, item, eui, alpha, lamb)
        
        #Adjust weights
        pu, qi = update_weights(pu, qi, user, item, eui, num_factors, alpha, lamb)
        
        
    return pu, qi, bu, bi

In [127]:
def minibatch_SGD(
    X, num_factors, n, m, mean_ratings, alpha=0.001, lamb=0.002):
    """
    Stochastic Gradiend Descent.

    Args:
      X: np.array
          ratings data 
      num_factor: int
          number of latent factors
      n: int
          number of unique users in ratings data
      m: int
          number of unique itens in ratings data
      alpha: float
          learning rate

    Returns: 

    pu: np.array
        user matrix factored
    qi: np.array
        item matrix factored
    bu: np.array
        bias vector for user weights
    bi: np.array
        bias vector for item weights
    """
    
    pu, qi, bu, bi = inicialization(n, m, num_factors)
    
    qi = qi.T
        
    for chunk in np.array_split(np.asarray(r), 10000):
        errors = 0
        for idx in range(chunk.shape[0]):
        
            user, item, rui = int(X[idx, 0]), int(X[idx, 1]), X[idx, 2]
            
            mean = mean_ratings.get(item)
            pred = one_svd_predict(pu, qi, bu, bi, user, item, num_factors, mean)
        
            #calculate error
            eui = rui - pred
            errors += eui
        
        for idx in range(chunk.shape[0]):
        
            bu, bi = update_bias(bu, bi, user, item, errors, alpha, lamb)
        
            #Adjust weights
            pu, qi = update_weights(pu, qi, user, item, errors, num_factors, alpha, lamb)
        
        
    return pu, qi, bu, bi

In [111]:
def fit(
    X_train, mean_ratings, num_factors, n, m, alpha=0.001, lamb=0.002, epochs=20, verbose=False):
    """
    Fit Stochastic Gradiend Descent.

    Args:
      X_train: np.array
          ratings data used to create the factored matrixes
      num_factor: int
          number of latent factors
      n: int
          number of unique users in ratings data
      m: int
          number of unique itens in ratings data
      alpha: float
          learning rate
      epochs: int
          number of steps 
      verbose: boolean
          if true, show error values at all steps of training

    Returns: 

    pu: np.array
        user matrix factored
    qi: np.array
        item matrix factored
    bu: np.array
        fitted user bias vector
    bi: np.array
        fitted item bias vector
    rmse: float
        root mean squared error between predict rating values and true ratings values
    mae: float
        mean absolute error between predict rating values and true ratings values
    """
    
    for epoch in range(epochs):
        
        #alpha = scheduler(epoch, alpha)
        
        pu, qi, bu, bi = SGD(X_train, num_factors, n, m, mean_ratings, alpha=alpha, lamb=lamb)
        
        y_pred, y_true = svd_predict(X_train, pu, qi, bu, bi, num_factors, mean_ratings)
        
        rmse = calc_rmse(y_pred, y_true)
        mae = calc_mae(y_pred, y_true)
        
        if rmse < 0.01:
            break
            
        if verbose:
            print("Epoch: {} - RMSE: {:.5f} - MAE: {:.5f}".format(epoch, rmse, mae))
            
    return pu, qi, bu, bi, rmse, mae

In [99]:
def create_df (df, users, items, user_column='UserId', item_column='ItemId'):
    """
    Creates a new rating dataframe where all users and itens are mapped for a continuous integer value
    and.

    Args:
      df: pd.DataFrame
          ratings_df
      user: list
          list with unique users code
      items: str
          list with unique items code
      user_column: str, defaul UserId
          column name of users
      item_column: str, default ItemId
          column name of items

    Returns: 

    df: 
        pandas DataFrame with all users items ratings
    dict_users: 
        dictionary mapped users and your new code
    dict_items: 
        dictionary mapped items and your new code     
    """
    
    dict_users = dict(zip(users, range(len(users))))
    dict_items = dict(zip(items, range(len(items))))

    df[user_column] = df[user_column].map(dict_users)
    df[item_column] = df[item_column].map(dict_items)

    df = df.fillna(-1)
    
    return np.asarray(df), df, dict_users, dict_items

In [100]:
def agg_ratings(df, map_dict, agg_metric='mean', agg_by='ItemId', rating_column='Rating'):
    """
    Aggregated ratings values by user or by items using metric mean or median.

    Args:
      df: pd.DataFrame
          ratings_df
      map_dict: dictionary
          dictionary of users or items created by create_df() function
      agg_metric: str, default 'mean'
          used metric to aggregate ratings
      agg_by: str, default 'ItemId'
          column name used do aggregate ratings.
          If aggregate column is ItemId, map_dict must be dict items.
      rating_column: str, default 'Rating'
          column name of ratings

    Returns: 

    mean_ratings: 
        dataframe with aggregated ratings
    dict_mean_ratings: 
        dictionary with aggregated ratings 
    """
    
    grouped_ratings = df.groupby(agg_by)
    
    if agg_metric == "mean":
        mean_ratings = grouped_ratings.mean().reset_index()
    elif agg_metric == "median":
        mean_ratings = grouped_ratings.median().reset_index()
        
    mean_ratings[agg_by] = mean_ratings[agg_by].map(map_dict)
    
    dict_mean_ratings = dict(zip(mean_ratings[agg_by], mean_ratings[rating_column]))
    
    return mean_ratings, dict_mean_ratings

In [101]:
def main_matrix_factorization(
    train_df, num_factors=100, alpha=0.001, lamb=0.002, epochs=20,
    agg_metric='mean', agg_by='ItemId', user_column='UserId',
    item_column='ItemId', rating_column='Rating', verbose=False):
    """
        1. Format df
        2. Matrix Factorization

    Args:
      X_train: np.array
          ratings data used to create the factored matrixes
      num_factors: int
          number of latent factors
      alpha: float
          learning rate
      lamb: float
          regularization factor
      epochs: int
          number of steps 
      user_column: str, defaul UserId
          column name of users
      item_column: str, default ItemId
          column name of items
      verbose: boolean, default False
          if true, show all steps

    Returns: 

    pu: np.array
        user matrix factored
    qi: np.array
        item matrix factored
    bu: np.array
        user bias vector
    bi: np.array
        item bias vector
    dict_users: 
        dictionary mapped users and your new code
    dict_items: 
        dictionary mapped items and your new code 
    r_df: pd.DataFrame
        Format data
    rmse: float
        root mean squared error between predict rating values and true ratings values
    mae: float
        mean absolute error between predict rating values and true ratings values
    """
    
    users = pd.unique(train_df[user_column]).tolist()
    items = pd.unique(train_df[item_column]).tolist()

    num_users = len(users)
    num_items = len(items)

    if verbose:
        print("\tFormatting data")
        
    r, r_df, dict_users, dict_items = create_df (
        train_df.copy(), users, items, user_column=user_column, item_column=item_column)

    if verbose:
        print("\tAggregating ratings")
        
    _, dict_mean_ratings = agg_ratings(
        train_df, dict_items, agg_metric=agg_metric, agg_by=agg_by, rating_column=rating_column)
      
    if verbose:
        print("\tFit SVD... ...\n")
        
    pu, qi, bu, bi, rmse, mae = fit(
        r, mean_ratings=dict_mean_ratings, num_factors=num_factors, n=num_users,
        m=num_items, alpha=alpha, lamb=lamb, epochs=epochs, verbose=verbose)
    
    return pu, qi, bu, bi, dict_users, dict_items, dict_mean_ratings, r_df, rmse, mae

In [146]:
def predict_one_rating(row, pu, qi, bu, bi, dict_users, dict_items, mean_ratings, agg_by="ItemId"):
    """
    Predict rating for a pair user-item

    Args:
      row: pd.Series
          line of pandas df
      pu: np.array
          user matrix
      qi: np.array
          item matrix
      bu: np.array
          user bias vector
      bi: np.array
          item bias vector
      user_mean: float
          mean ratings of all users

    Returns: 

    pred: float
        predicted value      
    """

    user_target = dict_users.get(row[0])
    item_target = dict_items.get(row[1])
    
    if agg_by == 'ItemId':
        user_mean = mean_ratings.get(item_target)
    elif agg_by == 'UserId':
        user_mean = mean_ratings.get(user_target)
        
    pred = np.dot(pu[user_target], qi[item_target]) + user_mean + bu[user_target] + bi[item_target]
    
    return pred

In [104]:
def predict_all_ratings(
    test,  mean_ratings, pu, qi, bu, bi, dict_users, dict_items,
    user_column='UserId', item_column='ItemId', rating_column='Rating', agg_by="ItemId"):
    """
    Predict ratings for all pair user-item in dataframe.

    Args:
      test: pd.DataFrame, columns default, [UserId, ItemId]
          train data
      user_mean: float
          mean ratings of all users
      train: pd.DataFrame, columns default, [UserId, ItemId, Ratings]
          train data
      pu: np.array
          user matrix
      qi: np.array
          item matrix
      bu: np.array
          user bias vector
      bi: np.array
          item bias vector
      dict_users: float
          mean ratings of all users
      dict_users: float
          mean ratings of all users
      user_column: str, defaul UserId
          column name of users
      item_column: str, default ItemId
          column name of items

    Returns: 

    DataFrame with all predict values     
    """
    
    test['ui'] = list(zip(test[user_column], test[item_column]))
    
    vfunc = np.vectorize(
        predict_one_rating,
        excluded=['pu', 'qi', 'bu', 'bi', 'dict_users', 'dict_items', 'mean_ratings', 'agg_by'])
    
    test[rating_column] = vfunc(
        row=test['ui'], pu=pu, qi=qi, bu=bu, bi=bi, dict_users=dict_users,
        dict_items=dict_items, mean_ratings=mean_ratings, agg_by=agg_by)
    
    return test[[user_column, item_column, rating_column]]

In [105]:
def round_(x):
    """
    Rounds the value to the integer more close.

    Args:
      x: float
          Value to be rounded
    Returns: 
        integer value   
    """
    
    return int(x + 0.5)

In [106]:
def get_mean_rating(df, agg_by='UserId'):
    
    grouped_df = df.groupby(agg_by)
    grouped_df = grouped_df.mean().reset_index()
    
    return grouped_df

In [107]:
def normalize_ratings(df, grouped_df, agg_by='UserId'):
    
    grouped_df = grouped_df.merge(df, on=agg_by)
    grouped_df['Rating'] =  grouped_df['Rating_y'] - grouped_df['Rating_x']  
    
    return grouped_df

In [108]:
def format_and_save(
    result, name_file='out.csv', user_column='UserId', item_column='ItemId',
    rating_column='Rating', round_preds=False):
    
    if round_preds:
        result[rating_column] = result[rating_column].apply(round_)
    else:
        result[rating_column] = result[rating_column]
        
    new_result = pd.DataFrame(np.vstack([result.columns, result]))
    text = list(new_result[0] + ":" + new_result[1] + "," + new_result[2].astype(str))
    
    result = "\n".join(text)
    
    return result
    

In [155]:
def main(
    train_file='ratings.csv', test_file='targets.csv', num_factors=40, alpha=0.01, lamb=0.02, epochs=1,
    agg_metric='mean', agg_by='ItemId', user_column='UserId', item_column='ItemId', rating_column='Rating',
    verbose=True):
    
    start_time = time.time()
    
    columns = '{}:{}'.format(user_column, item_column)
    
    if verbose:
        print("\n\nRead datasets")
        
    train = pd.read_csv(train_file)
    train[columns] = train[columns].str.split(':')
    train_df = pd.DataFrame(train[columns].to_list(), columns=[user_column, item_column])
    train_df[rating_column] = train[rating_column]

    test = pd.read_csv(test_file)
    test[columns] = test[columns].str.split(':')
    test_df = pd.DataFrame(test[columns].to_list(), columns=[user_column, item_column])
    
    grouped_df = get_mean_rating(train_df, agg_by=user_column)
    normalized_df = normalize_ratings(train_df, grouped_df, agg_by=user_column)
    
    train_df = normalized_df[[user_column, item_column, rating_column]]

    if verbose:
        print("Matrix Factorization")
        
    pu, qi, bu, bi, dict_users, dict_items, dict_mean_ratings, r_df, rmse, mae = main_matrix_factorization(
        train_df, num_factors=num_factors, alpha=alpha, epochs=epochs, lamb=lamb,
        agg_metric=agg_metric, verbose=verbose)
    qi = qi.T
    
    if verbose: 
        print("Predict Ratings")
        
    predictions = predict_all_ratings(
        test_df, dict_mean_ratings, pu, qi, bu, bi, dict_users=dict_users,
        dict_items=dict_items, user_column=user_column, item_column=item_column,
        rating_column=rating_column)
    
    predictions = predictions.merge(grouped_df, on=user_column)
    predictions[rating_column] = predictions['Rating_y'] + predictions['Rating_x']
    predictions = predictions[[user_column, item_column, rating_column]]
    
    predictions[rating_column] = predictions[rating_column].apply(clip)

    if verbose:
        print("Save Results")
        
    f = open('out.csv',"w")
    f.write(format_and_save(predictions.copy()))
    f.close()
    
    elapsed_time = (time.time() - start_time)/60
    print("Time executioon: {} minutes".format(elapsed_time))
    
    return pu, qi, bu, bi, dict_users, dict_items, dict_mean_ratings, predictions

In [156]:
pu, qi, bu, bi, dict_users, dict_items, dict_mean_ratings, predictions = main(
    train_file="data/ratings.csv", test_file='data/targets.csv')



Read datasets
Matrix Factorization
	Formatting data
	Aggregating ratings
	Fit SVD... ...

Epoch: 0 - RMSE: 0.56365 - MAE: 0.38112
Predict Ratings
Save Results
Time executioon: 2.069438846906026 minutes


In [157]:
predictions

,UserId,ItemId,Rating
0,33ce7ee122,34cb28c370,4.995804
1,eab9e065e5,34cb28c370,5.000000
2,f785763291,34cb28c370,4.647721
3,5f8185d75d,34cb28c370,5.000000
4,0eeef87507,1dfcdde662,4.960307
...,...,...,...
49625,e39df845bd,1fb1dc46b1,3.910221
49626,5cafd07f31,f556028715,4.621628
49627,7593b90096,43d5d80b1b,3.309567
49628,88a5a164b1,43d5d80b1b,4.080711


In [135]:
predictions = predictions.merge(grouped_df, on='UserId')
predictions['Rating'] = predictions['Rating_y'] + predictions['Rating_x']

,UserId,ItemId,Rating_x,Rating_y,Rating


In [137]:
predictions['Rating'] = predictions['Rating_y'] + predictions['Rating_x']

In [138]:
predictions['Rating']

0        6.0
1        6.0
2        5.5
3        6.0
4        6.0
        ... 
49625    5.0
49626    6.0
49627    5.0
49628    6.0
49629    6.0
Name: Rating, Length: 49630, dtype: float64

In [67]:
def fill_ratings_by_user(
   ratings_df, item_ratings, candidate_items, users, dict_users, dict_items,
    user_target, item_target, user_column, item_column):
    
    pred_ratings = {}
    
    rating

    for item in candidate_items:
        
        aux = {}
        
        for user in users:
            
            user_ratings = item_ratings.loc[item_ratings[user_column] == user]
            
            user_mean = user_mean = mean_ratings.get(item_target)
            
            rui = item_ratings.loc[item_ratings[item_column] == item]
            
            
    
            if not rui.empty:
                aux[user] = int(rui['Rating'])
            else:
                aux[user] = predict_one_rating(row, pu, qi, bu, bi, dict_users, dict_items, mean_ratings, agg_by="item")

        pred_ratings[item]= aux
        
    return pred_ratings

In [68]:
def fill_user_target_ratings(
    known_ratings, candidate_items, dict_users, dict_items, user_target, user_mean, item_column):
    
    pred_ratings = {}

    for j in range(1, len(candidate_items)):
    
        x = known_ratings.loc[known_ratings[item_column] == candidate_items[j]]
    
        if not x.empty:
            pred_ratings[candidate_items[j]] = int(x['Rating'])
        else:
            pred_ratings[candidate_items[j]] = clip(np.round(pu[dict_users.get(user_target)].dot(qi[dict_items.get(candidate_items[j])].T) + user_mean))
        
    return pred_ratings

In [69]:
def items_similarity():
    
    item_sim = {}
    target = aux_item[0] -user_mean
    
    for j in range(1, len(candidate_items)):
        
        candidate = aux_item[0] - user_mean
        
    item_sim[candidate_items[j]] = cosine_similarity(aux_item[0], aux_item[j]) 

In [70]:
def calculate_unknow_rating(user_target_ratings, item_sim, candidate_items):
    
    x = [np.dot(user_target_ratings[candidate_items[j]], item_sim[candidate_items[j]]) for j in range(1, len(candidate_items))]
    d = [np.abs(i) for i in item_sim.values()]

    pred = clip(np.round(sum(x)/sum(d)))
    
    return pred

In [71]:
def calculate_similarity(pred_ratings, item_target):

    item_target_vector = np.asarray(list(pred_ratings[item_target].values()))

    item_sim = {}

    for key in  pred_ratings.keys():
    
        if key != item_target:
            item_sim[key] =  cosine_similarity(item_target_vector, np.asarray(list(pred_ratings[key].values())))
            
    return item_sim

In [245]:
def item_based_predictv2(
    aux, ratings_df, pu, qi, dict_users, dict_items,
    n_neighboors, user_column, item_column):
    
  
    results = {'user': [], 'item': [], 'pred': []}

    for  idx, row in aux.iterrows():

        user_target = row[user_column]
        item_target = row[item_column]

        results['user'].append(user_target)
        results['item'].append(item_target)

        common_users = get_common_users_items(ratings_df, item=item_target)
        print(common_users)
        candidate_items = get_items_by_users(ratings_df, users=common_users)
        candidate_items.remove(item_target)
        candidate_items = candidate_items[:n_neighboors]
        candidate_items.insert(0, item_target)

        item_ratings = ratings_df.loc[ratings_df[item_column].isin(candidate_items)].reset_index(drop=True)
        
        pred_ratings = fill_ratings_by_user(
                ratings_df, item_ratings, candidate_items, common_users, dict_users,
                dict_items, user_target, user_column, item_column)

        item_sim = calculate_similarity(pred_ratings, item_target)

        #item_sim = {candidate_items[j]: cosine_similarity(qi[dict_items.get(candidate_items[0])], qi[dict_items.get(candidate_items[j])]) for j in range(1, len(candidate_items))}

        known_ratings = ratings_df.loc[ratings_df[user_column] == user_target].reset_index(drop=True)
        user_target_ratings = fill_user_target_ratings(
                known_ratings, candidate_items, dict_users, dict_items, user_target, user_mean, item_column)

        try:
            pred = calculate_unknow_rating(user_target_ratings, item_sim, candidate_items)
            results['pred'].append(pred)
            print(pred)
        except ZeroDivisionError:
            results['pred'].append(0)
            
    return results


In [13]:
name_train='ratings'
name_test='targets'
user_column='UserId'
item_column='ItemId'
rating_column='Rating'
path_to_read='data'
verbose=True
    
columns = '{}:{}'.format(user_column, item_column)
    
train = pd.read_csv("{}/{}.csv".format(path_to_read, name_train))
train[columns] = train[columns].str.split(':')
train_df = pd.DataFrame(train[columns].to_list(), columns=[user_column, item_column])
train_df[rating_column] = train[rating_column]

test = pd.read_csv("{}/{}.csv".format(path_to_read, name_test))
test[columns] = test[columns].str.split(':')
test_df = pd.DataFrame(test[columns].to_list(), columns=[user_column, item_column])

In [ ]:
agg_ratings(train_df, agg_metric='mean', agg_by='ItemId', rating_column='Rating')

In [ ]:
item_based_predictv2(
    aux, ratings_df, pu, qi, dict_users, dict_items,
    n_neighboors, user_column, item_column)

In [75]:
test_df

,UserId,ItemId
0,33ce7ee122,34cb28c370
1,eab9e065e5,34cb28c370
2,f785763291,34cb28c370
3,5f8185d75d,34cb28c370
4,0eeef87507,1dfcdde662
...,...,...
49625,435f4de184,8dba6546ea
49626,7593b90096,43d5d80b1b
49627,88a5a164b1,43d5d80b1b
49628,90df18a22b,58c3b27fe2


In [112]:
user_target='33ce7ee122'
item_target='34cb28c370'

In [114]:
n_neighboors=3

In [125]:
common_users = get_common_users_items(train_df, item=item_target)

candidate_items = get_items_by_users(train_df, users=common_users)
candidate_items.remove(item_target)
candidate_items = candidate_items
candidate_items.insert(0, item_target)

In [126]:
item_ratings = train_df.loc[train_df[item_column].isin(candidate_items)].reset_index(drop=True)

In [127]:
item_ratings = item_ratings.reset_index(drop=True)

In [128]:
user_target='33ce7ee122'
item_target='34cb28c370'

common_users = get_common_users_items(train_df, item=item_target)

candidate_items = get_items_by_users(train_df, users=common_users)
candidate_items.remove(item_target)
candidate_items = candidate_items
candidate_items.insert(0, item_target)

item_ratings = item_ratings.reset_index(drop=True)
item_ratings = train_df.loc[train_df[item_column].isin(candidate_items)].reset_index(drop=True)
users_df = pd.DataFrame({'UserId': pd.unique(train_df['UserId'])})
items_df = pd.DataFrame({'ItemId': pd.unique(item_ratings['ItemId'])})

df = pd.merge(users_df, items_df, how="cross")
df = df.merge(item_ratings, on=["UserId", 'ItemId'], how='left')

In [129]:
df = pd.merge(users_df, items_df, how="cross")
df = df.merge(item_ratings, on=["UserId", 'ItemId'], how='left')
df

,UserId,ItemId,Rating
0,4baf0ac888,34cb28c370,NaN
1,4baf0ac888,a33ba0df95,NaN
2,4baf0ac888,8cbeb61a98,NaN
3,4baf0ac888,8991f778ae,NaN
4,4baf0ac888,471364d9d8,NaN
...,...,...,...
4846819,d487cff9fc,84d21582e7,NaN
4846820,d487cff9fc,7dd53de0b4,NaN
4846821,d487cff9fc,34ba4d3544,NaN
4846822,d487cff9fc,66a95c9594,NaN


In [130]:
df = df.fillna(0)

In [98]:
dict_mean_ratings = dict(zip([agg_by], mean_ratings[rating_column]))

In [ ]:
dict_users, dict_items, dict_mean_ratings

In [90]:
mean_target = dict_mean_ratings.get(dict_items.get(item_target))

In [95]:
df.loc[(df['ItemId']==item_target) & (df['Rating'].isna()) , 'Rating'] = mean_target

In [148]:
aux = merged.loc[merged['Rating'].isna()]

In [121]:
for item_target in candidate_items:
    
    mean_target = dict_mean_ratings.get(dict_items.get(item_target))
    df.loc[(df['ItemId'] == item_target) & (df['Rating'].isna()) , 'Rating'] = mean_target

In [143]:
target = df.loc[df['ItemId'] == item_target]['Rating']
neighbor = df.loc[~df['ItemId'] == item_target]
similarities = {}

for item in candidate_items[1:]:
    
    neighbor = df.loc[df['ItemId'] == item]['Rating']
    similarities[item] = cosine_similarity(target, neighbor)

In [159]:
target = df.loc[df['ItemId'] == item_target]['Rating']
neighbor = df.loc[df['ItemId'] != item_target]

grouped_df = neighbor.groupby('ItemId')

In [184]:
def calculate_similarity(df, item_target):

    target = df.loc[df['ItemId'] == item_target]['Rating']
    neighbor = df.loc[df['ItemId'] != item_target]

    grouped_df = neighbor.groupby('ItemId')
                                  
    similarities = {}                              

    for name, group in grouped_df:

        similarities[name]= cosine_similarity(target, group['Rating'])
    
    return similarities

In [185]:
def fill_user_target_ratings(
    user_target, candidate_items, user_df, pu, qi, bu, bi, dict_users, dict_items, dict_mean_ratings, agg_by="ItemId"):
    
    ratings = {}

    for item in candidate_items[1:]:
        
        aux = user_df.loc[user_df['ItemId'] == item]

        if aux.empty:
            pred = predict_one_rating(
            [user_target, item], pu, qi, bu, bi, dict_users, dict_items, dict_mean_ratings, agg_by="ItemId")
        else:
            pred = aux['Rating']
        
        ratings[item] = pred
        
    return ratings

In [181]:
def neighbor_predict(user_target_ratings, similarities, candidate_items):
    
    numerador = 0
    denominador = sum(list(map(abs, similarities.values())))

    for item in candidate_items[1:]:
        numerador += user_target_ratings.get(item) * similarities.get(item)

    return numerador/denominador

In [254]:
def test(
    test_df, train_df, pu, qi, bu, bi, dict_users, dict_items, dict_mean_ratings,
    n_neighboors=3, user_column='UserId', item_column='ItemId'):
    
    results = {'UserId': [], 'ItemId': [], 'Rating': []}

    start_time = time.time()

    for  idx, row in test_df.iterrows():

        user_target = row[user_column]
        item_target = row[item_column]

        results['UserId'].append(user_target)
        results['UserId'].append(item_target)

        common_users = get_common_users_items(train_df, item=item_target)

        candidate_items = get_items_by_users(train_df, users=common_users)
        candidate_items.remove(item_target)
        candidate_items = candidate_items[: n_neighboors]
        candidate_items.insert(0, item_target)

        if len(candidate_items) != 1:
            
            item_ratings = train_df.loc[train_df[item_column].isin(candidate_items)].reset_index(drop=True)

            users_df = pd.DataFrame({'UserId': pd.unique(train_df['UserId'])})
            items_df = pd.DataFrame({'ItemId': pd.unique(item_ratings['ItemId'])})

            df = pd.merge(users_df, items_df, how="cross")
            df = df.merge(item_ratings, on=["UserId", 'ItemId'], how='left')
            df = df.fillna(0)

            similarities = calculate_similarity(df, item_target)

            user_df = train_df.loc[train_df['UserId'] == user_target]

            user_target_ratings = fill_user_target_ratings(
                user_target, candidate_items, user_df, pu, qi, bu, bi, dict_users, dict_items, dict_mean_ratings, agg_by="ItemId")

            results['Rating'] = neighbor_predict(user_target_ratings, similarities, candidate_items)
    
        else:
            results['Rating'] = user_df = train_df.loc[train_df['UserId'] == user_target]['Rating'].mean()

    elapsed_time = (time.time() - start_time)/60
    print("Time executioon: {} minutes".format(elapsed_time))
    
    return results


In [ ]:
preds = test(test_df, train_df, pu, qi, bu, bi, dict_users, dict_items, dict_mean_ratings)

In [31]:
grouped_ratings = train_df.groupby('UserId')
grouped_ratings = grouped_ratings.mean().reset_index()

In [32]:
grouped_users = train_df.groupby('UserId')
grouped_users

In [66]:
def get_mean_rating(df, agg_by='UserId'):
    
    grouped_df = df.groupby(agg_by)
    grouped_df = grouped_df.mean().reset_index()
    
    return grouped_df

def normalize_ratings(df, grouped_df, agg_by='UserId'):
    
    grouped_df = grouped_df.merge(df, on=agg_by)
    grouped_df['Rating'] =  grouped_df['Rating_y'] - grouped_df['Rating_x']  
    
    return grouped_df

In [64]:
def normalize_ratings(df, grouped_df, agg_by='UserId'):
    
    grouped_df = grouped_df.merge(df, on=agg_by)
    grouped_df['Rating'] =  grouped_df['Rating_y'] - grouped_df['Rating_x']  
    
    return grouped_df

In [68]:
grouped_df = get_mean_rating(train_df, agg_by='UserId')
train_df = normalize_ratings(train_df, grouped_df, agg_by='UserId')

In [69]:
train_df = normalize_ratings(train_df, grouped_df, agg_by='UserId')

In [ ]:
train_df = normalize_ratings(df, agg_by='UserId')
targets_df = normalize_ratings(df, agg_by='UserId')

In [123]:
train_df.loc[train_df['UserId']== None]

,UserId,Rating_x,ItemId,Rating_y,Rating


In [55]:
grouped_ratings['Rating'] =  grouped_ratings['Rating_y'] - grouped_ratings['Rating_x']  

In [70]:
train_df

,UserId,Rating_x,ItemId,Rating_y,Rating
0,0000070c95,5.0,a7aabc2903,5,0.0
1,000058232b,1.5,8f6500d75b,1,-0.5
2,000058232b,1.5,b7c3b102ef,2,0.5
3,000157812f,5.0,a4232bb20c,5,0.0
4,0001a428de,3.0,7bda68c619,1,-2.0
...,...,...,...,...,...
403209,ffffbeb8e0,4.0,45f9749698,4,0.0
403210,ffffcdee5f,5.0,9b6dc780c7,5,0.0
403211,ffffec2967,5.0,0584dddf72,5,0.0
403212,ffffec2967,5.0,d4679eb6bd,5,0.0


In [60]:
grouped_ratings.loc[grouped_ratings['UserId'] == 'ffffec2967']

,UserId,Rating_x,ItemId,Rating_y,Rating
403211,ffffec2967,5.0,0584dddf72,5,0.0
403212,ffffec2967,5.0,d4679eb6bd,5,0.0


In [51]:
list_df = []

for idx, group in grouped_users:

    mean_user = dict_mean_ratings.get(idx)
    
    group['Rating'] =  group['Rating'] - mean_user

    list_df.append(group)


KeyboardInterrupt: 

In [47]:
dict_mean_ratings = dict(zip(grouped_ratings['UserId'], grouped_ratings['Rating']))

In [50]:
pd.concat(list_df)

[            UserId      ItemId  Rating
 189772  3360f21e5e  38a212776b       5,
             UserId      ItemId  Rating
 189772  3360f21e5e  38a212776b       5,
             UserId      ItemId  Rating
 189772  3360f21e5e  38a212776b       5,
             UserId      ItemId  Rating
 189772  3360f21e5e  38a212776b       5,
             UserId      ItemId  Rating
 189772  3360f21e5e  38a212776b       5,
             UserId      ItemId  Rating
 189772  3360f21e5e  38a212776b       5,
             UserId      ItemId  Rating
 189772  3360f21e5e  38a212776b       5,
             UserId      ItemId  Rating
 189772  3360f21e5e  38a212776b       5,
             UserId      ItemId  Rating
 189772  3360f21e5e  38a212776b       5,
             UserId      ItemId  Rating
 189772  3360f21e5e  38a212776b       5,
             UserId      ItemId  Rating
 189772  3360f21e5e  38a212776b       5,
             UserId      ItemId  Rating
 189772  3360f21e5e  38a212776b       5,
             UserId      Ite

In [216]:
candidate_items = get_items_by_users(train_df, users=common_users)
print(candidate_items)
candidate_items.remove('7724ef1f69')
candidate_items = candidate_items[: 3]
candidate_items.insert(0, '7724ef1f69')
candidate_items

['3dbe2da2b7', '55f95f8426', '6b652e3a14', '7724ef1f69', '92055fd567']


['7724ef1f69', '3dbe2da2b7', '55f95f8426', '6b652e3a14']

In [207]:
users_df = pd.DataFrame({'UserId': pd.unique(train_df['UserId'])})
items_df = pd.DataFrame({'ItemId': pd.unique(item_ratings['ItemId'])})

df = pd.merge(users_df, items_df, how="cross")
df = df.merge(item_ratings, on=["UserId", 'ItemId'], how='left')
df = df.fillna(0)
df

,UserId,ItemId,Rating
0,4baf0ac888,7724ef1f69,0.0
1,4baf0ac888,3dbe2da2b7,0.0
2,4baf0ac888,6b652e3a14,0.0
3,4baf0ac888,55f95f8426,0.0
4,4baf0ac888,92055fd567,0.0
...,...,...,...
1346335,d487cff9fc,7724ef1f69,0.0
1346336,d487cff9fc,3dbe2da2b7,0.0
1346337,d487cff9fc,6b652e3a14,0.0
1346338,d487cff9fc,55f95f8426,0.0


In [172]:
similarities = calculate_similarity(df, item_target)

In [166]:
user_target_ratings = fill_user_target_ratings(
    user_target, candidate_items, pu, qi, bu, bi, dict_users, dict_items, dict_mean_ratings, agg_by="ItemId")

In [176]:
neighbor_predict(user_target_ratings, similarities, candidate_items)

4.009925780584842

In [260]:
users = pd.unique(train_df[user_column]).tolist()
items = pd.unique(train_df[item_column]).tolist()

num_users = len(users)
num_items = len(items)

r, r_df, dict_users, dict_items = create_df (
        train_df.copy(), users, items, user_column=user_column, item_column=item_column)

In [268]:
def minibatch_SGD(
    X, num_factors, n, m, mean_ratings, alpha=0.001, lamb=0.002):
    """
    Stochastic Gradiend Descent.

    Args:
      X: np.array
          ratings data 
      num_factor: int
          number of latent factors
      n: int
          number of unique users in ratings data
      m: int
          number of unique itens in ratings data
      alpha: float
          learning rate

    Returns: 

    pu: np.array
        user matrix factored
    qi: np.array
        item matrix factored
    bu: np.array
        bias vector for user weights
    bi: np.array
        bias vector for item weights
    """
    
    pu, qi, bu, bi = inicialization(n, m, num_factors)
    
    qi = qi.T
        
    for chunk in np.array_split(np.asarray(r), 10000):
        errors = 0
        for idx in range(chunk.shape[0]):
        
            user, item, rui = X[idx, 0], X[idx, 1], X[idx, 2]
            mean = mean_ratings.get(item)
            pred = one_svd_predict(pu, qi, bu, bi, user, item, num_factors, mean)
        
            #calculate error
            eui = rui - pred
            errors += eui
        
        for idx in range(chunk.shape[0]):
        
            bu, bi = update_bias(bu, bi, user, item, errors, alpha, lamb)
        
            #Adjust weights
            pu, qi = update_weights(pu, qi, user, item, errors, num_factors, alpha, lamb)
        
        
    return pu, qi, bu, bi

In [266]:
X = r


for chunk in np.array_split(np.asarray(r), 10000):
    errors = 0
    for idx in range(chunk.shape[0]):
        
        user, item, rui = X[idx, 0], X[idx, 1], X[idx, 2]
        mean = mean_ratings.get(item)
        pred = one_svd_predict(pu, qi, bu, bi, user, item, num_factors, mean)
        
        #calculate error
        eui = rui - pred
        errors += eui
        
    for idx in range(chunk.shape[0]):
        
        bu, bi = update_bias(bu, bi, user, item, eui, alpha, lamb)
        
        #Adjust weights
        pu, qi = update_weights(pu, qi, user, item, eui, num_factors, alpha, lamb)

(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)


(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)


(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)
(41, 3)


(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)


(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)


(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)


(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)


(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
(40, 3)
